In [2]:
import os
import torch
import pandas as pd
from tqdm import tqdm
from PIL import Image
import numpy as np
import cv2
from pathlib import Path
from transformers import AutoImageProcessor, SiglipForImageClassification

# =========================================
# 1️⃣ CONFIGURATION
# =========================================
MODEL_NAME = "prithivMLmods/Realistic-Gender-Classification"
INPUT_DIR = Path("../data/crops_face/20251107/5_cafe_pos_faces_unique")
OUTPUT_CSV = Path("../data/predictions/cafe_pos/20251107/gender_annotated/gender_predictions.csv")
ANNOTATION_DIR = Path("../data/predictions/cafe_pos/20251107/gender_annotated")
ANNOTATION_DIR.mkdir(parents=True, exist_ok=True)

FONT = cv2.FONT_HERSHEY_SIMPLEX
device = "cuda" if torch.cuda.is_available() else "cpu"

# =========================================
# 2️⃣ LOAD MODEL
# =========================================
print(f"📦 Loading model: {MODEL_NAME}")
processor = AutoImageProcessor.from_pretrained(MODEL_NAME)
model = SiglipForImageClassification.from_pretrained(MODEL_NAME).to(device)
model.eval()
print("✅ Model loaded successfully.\n")

# ID → Label mapping
id2label = {
    "0": "Female",
    "1": "Male"
}

# =========================================
# 3️⃣ GENDER PREDICTION FUNCTION
# =========================================
def classify_gender(image_path):
    """
    Classifies gender using SigLIP2-based model.
    Returns (gender_label, confidence_score)
    """
    try:
        image = Image.open(image_path).convert("RGB")
        inputs = processor(images=image, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1).squeeze().cpu().numpy()

        pred_id = int(np.argmax(probs))
        gender = id2label[str(pred_id)]
        confidence = float(probs[pred_id])

        return gender, confidence
    except Exception as e:
        print(f"⚠️ Error processing {image_path}: {e}")
        return "Unknown", 0.0

# =========================================
# 4️⃣ PROCESS IMAGES
# =========================================
images = sorted([f for f in INPUT_DIR.glob("*") if f.suffix.lower() in [".jpg", ".jpeg", ".png"]])
print(f"🧠 Found {len(images)} face images to process...\n")

results = []

for img_path in tqdm(images, desc="Predicting gender (SigLIP2)"):
    gender, conf = classify_gender(img_path)

    img = cv2.imread(str(img_path))
    if img is None:
        continue

    label = f"{gender} ({conf*100:.1f}%)"
    overlay = img.copy()
    cv2.rectangle(overlay, (0, 0), (img.shape[1], 50), (0, 0, 0), -1)
    annotated = cv2.addWeighted(overlay, 0.4, img, 0.6, 0)
    cv2.putText(annotated, label, (15, 35), FONT, 0.9, (255, 255, 255), 2, cv2.LINE_AA)

    out_path = ANNOTATION_DIR / f"{img_path.stem}_gender.jpg"
    cv2.imwrite(str(out_path), annotated, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

    results.append({
        "file": img_path.name,
        "predicted_gender": gender,
        "confidence": round(conf, 3)
    })

# =========================================
# 5️⃣ SAVE RESULTS
# =========================================
df = pd.DataFrame(results)
df.to_csv(OUTPUT_CSV, index=False)

print(f"\n✅ Saved {len(df)} results to {OUTPUT_CSV}")
print(f"📸 Annotated images saved to {ANNOTATION_DIR}")
df.head()


📦 Loading model: prithivMLmods/Realistic-Gender-Classification
✅ Model loaded successfully.

🧠 Found 235 face images to process...



Predicting gender (SigLIP2): 100%|██████████| 235/235 [01:36<00:00,  2.42it/s]


✅ Saved 235 results to ..\data\predictions\cafe_pos\20251107\gender_annotated\gender_predictions.csv
📸 Annotated images saved to ..\data\predictions\cafe_pos\20251107\gender_annotated


,file,predicted_gender,confidence
0,cluster_000_AccoID_1170854_20251107_091623.png,Female,0.999
1,cluster_001_AccoID_1171262_20251107_112821.png,Female,0.970
2,cluster_002_AccoID_1171659_20251107_135038.png,Female,0.948
3,cluster_004_AccoID_1171659_20251107_135011.png,Female,0.911
4,cluster_006_AccoID_1171574_20251107_132453.png,Male,0.785


In [15]:
import pandas as pd

# Paths
gender_csv = "../data/predictions/cafe_pos/20251105/gender_annotated/gender_predictions.csv"
results_stage1 = "../results/20251105/results_faces_accounting_20251105.csv"
output_csv = "../results/20251105/results_faces_accounting_20251105.csv"

# Load data
gender_df = pd.read_csv(gender_csv)
results_df = pd.read_csv(results_stage1)

print("✅ Loaded files:")
print(f" - Gender predictions: {len(gender_df)} rows")
print(f" - Stage 1 results: {len(results_df)} rows")


✅ Loaded files:
 - Gender predictions: 287 rows
 - Stage 1 results: 796 rows


In [16]:
# Ensure same column naming for merge
gender_df.rename(columns={"file": "image_name"}, inplace=True)

# Clean up naming (in case of directory paths)
gender_df["image_name"] = gender_df["image_name"].apply(lambda x: x.split("/")[-1])
results_df["image_name"] = results_df["image_name"].apply(lambda x: x.split("/")[-1])


In [17]:
merged_df = results_df.merge(
    gender_df,
    on="image_name",
    how="left"
)


In [18]:
merged_df.to_csv(output_csv, index=False)
print(f"✅ Stage 2 results with gender saved to: {output_csv}")


✅ Stage 2 results with gender saved to: ../results/20251105/results_faces_accounting_20251105.csv
